# Описание
Этот ноутбук содержит решение первого домашнего задания "Оценки и доверительные интервалы"

## Результаты


## Материалы:
* [Условие задач](mspsdz1.pdf)
* [Первая лекция](MSPS_2020_1.pdf)

# Setup

## Library import
We import all the required Python libraries

In [2]:
# Data manipulation
import pandas as pd
import numpy as np

import matplotlib as plt